In [1]:
!git clone https://github.com/Rnlcksgdkd/test_GAN

Cloning into 'test_GAN'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 165 (delta 4), reused 6 (delta 2), pack-reused 153
Receiving objects: 100% (165/165), 92.36 MiB | 22.54 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Checking out files: 100% (111/111), done.


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 50.1MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 460kB 29.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import re
import collections
from konlpy.tag import Twitter, Kkma
import pickle
import collections
import random
import numpy as np
import os
import time
import argparse

import math
import copy

from gensim.models.word2vec import Word2Vec

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

> ## Gen_Data_Loader()

In [5]:
class Gen_Data_loader():
    def __init__(self, batch_size, sen_length):
        self.batch_size = batch_size
        self.token_stream = []
        self.sen_length = sen_length

    def create_batches(self, data_file):
        self.token_stream = []
        with open(data_file, 'r') as f:
            for line in f:
                line = line.strip()
                line = line.split()
                parse_line = [int(x) for x in line]
                if len(parse_line) == self.sen_length:
                    self.token_stream.append(parse_line)

        self.num_batch = int(len(self.token_stream) / self.batch_size)
        self.token_stream = self.token_stream[:self.num_batch * self.batch_size]
        self.sequence_batch = np.split(np.array(self.token_stream), self.num_batch, 0)
        self.pointer = 0

    def next_batch(self):
        ret = self.sequence_batch[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0


class Dis_dataloader():
    def __init__(self, batch_size, sen_length):
        self.batch_size = batch_size
        self.sentences = np.array([])
        self.labels = np.array([])
        self.sen_length = sen_length

    def load_train_data(self, positive_file, negative_file):
        # Load data
        positive_examples = []
        negative_examples = []
        with open(positive_file)as fin:
            for line in fin:
                line = line.strip()
                line = line.split()
                parse_line = [int(x) for x in line]
                positive_examples.append(parse_line)
        with open(negative_file)as fin:
            for line in fin:
                line = line.strip()
                line = line.split()
                parse_line = [int(x) for x in line]
                if len(parse_line) == self.sen_length:
                    negative_examples.append(parse_line)
        self.sentences = np.array(positive_examples + negative_examples)

        # Generate labels
        positive_labels = [[0, 1] for _ in positive_examples]
        negative_labels = [[1, 0] for _ in negative_examples]
        self.labels = np.concatenate([positive_labels, negative_labels], 0)

        # Shuffle the data
        shuffle_indices = np.random.permutation(np.arange(len(self.labels)))
        self.sentences = self.sentences[shuffle_indices]
        self.labels = self.labels[shuffle_indices]

        # Split batches
        self.num_batch = int(len(self.labels) / self.batch_size)
        self.sentences = self.sentences[:self.num_batch * self.batch_size]
        self.labels = self.labels[:self.num_batch * self.batch_size]
        self.sentences_batches = np.split(self.sentences, self.num_batch, 0)
        self.labels_batches = np.split(self.labels, self.num_batch, 0)

        self.pointer = 0


    def next_batch(self):
        ret = self.sentences_batches[self.pointer], self.labels_batches[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0

> ## Generator

In [6]:

class Generator(nn.Module):
    """ Generator """

    def __init__(self, vocab_size, embedding_dim, hidden_dim, use_cuda):
        super(Generator, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_cuda = use_cuda
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.init_params()

    def forward(self, x):
        """
        Embeds input and applies LSTM on the input sequence.
        Inputs: x
            - x: (batch_size, seq_len), sequence of tokens generated by generator
        Outputs: out
            - out: (batch_size * seq_len, vocab_size), lstm output prediction
        """
        self.lstm.flatten_parameters()
        h0, c0 = self.init_hidden(x.size(0))
        emb = self.embed(x) # batch_size * seq_len * emb_dim 
        out, _ = self.lstm(emb, (h0, c0)) # out: batch_size * seq_len * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # (batch_size*seq_len) * vocab_size
        return out

    def step(self, x, h, c):
        """
        Embeds input and applies LSTM one token at a time (seq_len = 1).
        Inputs: x, h, c
            - x: (batch_size, 1), sequence of tokens generated by generator
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state
        Outputs: out, h, c
            - out: (batch_size, vocab_size), lstm output prediction
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state 
        """
        self.lstm.flatten_parameters()
        emb = self.embed(x) # batch_size * 1 * emb_dim
        out, (h, c) = self.lstm(emb, (h, c)) # out: batch_size * 1 * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # batch_size * vocab_size
        return out, h, c

    def init_hidden(self, batch_size):
        h = torch.zeros(1, batch_size, self.hidden_dim)
        c = torch.zeros(1, batch_size, self.hidden_dim)
        if self.use_cuda:
            h, c = h.cuda(), c.cuda()
        return h, c
    
    def init_params(self):
        for param in self.parameters():
            param.data.uniform_(-0.05, 0.05)

    def sample(self, batch_size, seq_len, x=None):
        """
        Samples the network and returns a batch of samples of length seq_len.
        Outputs: out
            - out: (batch_size * seq_len)
        """
        samples = []
        if x is None:
            h, c = self.init_hidden(batch_size)
            x = torch.zeros(batch_size, 1, dtype=torch.int64)
            if self.use_cuda:
                x = x.cuda()
            for _ in range(seq_len):
                out, h, c = self.step(x, h, c)
                prob = torch.exp(out)
                x = torch.multinomial(prob, 1)
                samples.append(x)
        else:
            h, c = self.init_hidden(x.size(0))
            given_len = x.size(1)
            lis = x.chunk(x.size(1), dim=1)
            for i in range(given_len):
                out, h, c = self.step(lis[i], h, c)
                samples.append(lis[i])
            prob = torch.exp(out)
            x = torch.multinomial(prob, 1)
            for _ in range(given_len, seq_len):
                samples.append(x)
                out, h, c = self.step(x, h, c)
                prob = torch.exp(out)
                x = torch.multinomial(prob, 1)
        out = torch.cat(samples, dim=1) # along the batch_size dimension
        return out

        

# Discriminator

In [ ]:

# a = open('./data/pretrain_embedding_vec.pkl', 'rb')
# word_embedding_matrix = pickle.load(a)
# word_embedding_matrix = word_embedding_matrix.astype(np.float32)

# An alternative to tf.nn.rnn_cell._linear function, which has been removed in Tensorfow 1.0.1
# The highway layer is borrowed from https://github.com/mkroutikov/tf-lstm-char-cnn

class Discriminator(nn.Module):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    Highway architecture based on the pooled feature maps is added. Dropout is adopted.
    """
    def __init__(self, num_classes, vocab_size, embedding_dim, filter_sizes, num_filters, dropout_prob):
        super(Discriminator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_f, (f_size, embedding_dim)) for f_size, num_f in zip(filter_sizes, num_filters)
        ])
        self.highway = nn.Linear(sum(num_filters), sum(num_filters))
        self.dropout = nn.Dropout(p = dropout_prob)
        self.fc = nn.Linear(sum(num_filters), num_classes)

    def forward(self, x):
        """
        Inputs: x
            - x: (batch_size, seq_len)
        Outputs: out
            - out: (batch_size, num_classes)
        """
        emb = self.embed(x).unsqueeze(1) # batch_size, 1 * seq_len * emb_dim
        convs = [F.relu(conv(emb)).squeeze(3) for conv in self.convs] # [batch_size * num_filter * seq_len]
        pools = [F.max_pool1d(conv, conv.size(2)).squeeze(2) for conv in convs] # [batch_size * num_filter]
        out = torch.cat(pools, 1)  # batch_size * sum(num_filters)
        highway = self.highway(out)
        transform = F.sigmoid(highway)
        out = transform * F.relu(highway) + (1. - transform) * out # sets C = 1 - T
        out = F.log_softmax(self.fc(self.dropout(out)), dim=1) # batch * num_classes
        return out

        

> ## TargetLSTM

In [ ]:

class TargetLSTM(nn.Module):
    """ Target LSTM """

    def __init__(self,  vocab_size, embedding_dim, hidden_dim, use_cuda):
        super(TargetLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_cuda = use_cuda
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.init_params()

    def forward(self, x):
        """
        Embeds input and applies LSTM on the input sequence.
        Inputs: x
            - x: (batch_size, seq_len), sequence of tokens generated by generator
        Outputs: out
            - out: (batch_size, vocab_size), lstm output prediction
        """
        self.lstm.flatten_parameters()
        h0, c0 = self.init_hidden(x.size(0))
        emb = self.embed(x) # batch_size * seq_len * emb_dim 
        out, _ = self.lstm(emb, (h0, c0)) # out: seq_len * batch_size * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # seq_len * batch_size * vocab_size
        return out

    def step(self, x, h, c):
        """
        Embeds input and applies LSTM one token at a time (seq_len = 1).
        Inputs: x, h, c
            - x: (batch_size, 1), sequence of tokens generated by generator
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state
        Outputs: out, h, c
            - out: (batch_size, 1, vocab_size), lstm output prediction
            - h: (1, batch_size, hidden_dim), lstm hidden state
            - c: (1, batch_size, hidden_dim), lstm cell state 
        """
        self.lstm.flatten_parameters()
        emb = self.embed(x) # batch_size * 1 * emb_dim
        out, (h, c) = self.lstm(emb, (h, c)) # out: batch_size * 1 * hidden_dim
        out = self.log_softmax(self.fc(out.contiguous().view(-1, self.hidden_dim))) # batch_size * vocab_size
        return out, h, c

    def init_hidden(self, batch_size):
        h = torch.zeros((1, batch_size, self.hidden_dim))
        c = torch.zeros((1, batch_size, self.hidden_dim))
        if self.use_cuda:
            h, c = h.cuda(), c.cuda()
        return h, c
    
    def init_params(self):
        for param in self.parameters():
            param.data.normal_(0, 1)

    def sample(self, batch_size, seq_len):
        """
        Samples the network and returns a batch of samples of length seq_len.
        Outputs: out
            - out: (batch_size * seq_len)
        """
        samples = []
        h, c = self.init_hidden(batch_size)
        x = torch.zeros(batch_size, 1, dtype=torch.int64)
        if self.use_cuda:
            x = x.cuda()
        for _ in range(seq_len):
            out, h, c = self.step(x, h, c)
            prob = torch.exp(out)
            x = torch.multinomial(prob, 1)
            samples.append(x)
        out = torch.cat(samples, dim=1) # along the batch_size dimension
        return out

> ## PGLoss 

In [ ]:
class PGLoss(nn.Module):
    """
    Pseudo-loss that gives corresponding policy gradients (on calling .backward()) 
    for adversial training of Generator
    """

    def __init__(self):
        super(PGLoss, self).__init__()

    def forward(self, pred, target, reward):
        """
        Inputs: pred, target, reward
            - pred: (batch_size, seq_len), 
            - target : (batch_size, seq_len), 
            - reward : (batch_size, ), reward of each whole sentence
        """
        one_hot = torch.zeros(pred.size(), dtype=torch.uint8)
        if pred.is_cuda:
            one_hot = one_hot.cuda()
        one_hot.scatter_(1, target.data.view(-1, 1), 1)
        loss = torch.masked_select(pred, one_hot)
        loss = loss * reward.contiguous().view(-1)
        loss = -torch.sum(loss)
        return loss

> ## data iter

In [ ]:

class GenDataIter:
    """ Toy data iter to load digits """

    def __init__(self, data_file, batch_size):
        super(GenDataIter, self).__init__()
        self.batch_size = batch_size
        self.data_lis = self.read_file(data_file)
        self.data_num = len(self.data_lis)
        self.indices = range(self.data_num)
        self.num_batches = math.ceil(self.data_num / self.batch_size)
        self.idx = 0
        self.reset()

    def __len__(self):
        return self.num_batches

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def reset(self):
        self.idx = 0
        random.shuffle(self.data_lis)

    def next(self):
        if self.idx >= self.data_num:
            raise StopIteration
        index = self.indices[self.idx : self.idx + self.batch_size]
        d = [self.data_lis[i] for i in index]
        d = torch.tensor(d)

        # 0 is prepended to d as start symbol
        data = torch.cat([torch.zeros(len(index), 1, dtype=torch.int64), d], dim=1)
        target = torch.cat([d, torch.zeros(len(index), 1, dtype=torch.int64)], dim=1)
        
        self.idx += self.batch_size
        return data, target

    def read_file(self, data_file):
        with open(data_file, 'r') as f:
            lines = f.readlines()
        lis = []
        for line in lines:
            l = [int(s) for s in list(line.strip().split())]
            lis.append(l)
        return lis


class DisDataIter:
    """ Toy data iter to load digits """

    def __init__(self, real_data_file, fake_data_file, batch_size):
        super(DisDataIter, self).__init__()
        self.batch_size = batch_size
        real_data_lis = self.read_file(real_data_file)
        fake_data_lis = self.read_file(fake_data_file)
        self.data = real_data_lis + fake_data_lis
        self.labels = [1 for _ in range(len(real_data_lis))] +\
                        [0 for _ in range(len(fake_data_lis))]
        self.pairs = list(zip(self.data, self.labels))
        self.data_num = len(self.pairs)
        self.indices = range(self.data_num)
        self.num_batches = math.ceil(self.data_num / self.batch_size)
        self.idx = 0
        self.reset()

    def __len__(self):
        return self.num_batches

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def reset(self):
        self.idx = 0
        random.shuffle(self.pairs)

    def next(self):
        if self.idx >= self.data_num:
            raise StopIteration
        index = self.indices[self.idx : self.idx + self.batch_size]
        pairs = [self.pairs[i] for i in index]
        data = [p[0] for p in pairs]
        label = [p[1] for p in pairs]
        data = torch.tensor(data)
        label = torch.tensor(label)
        self.idx += self.batch_size
        return data, label

    def read_file(self, data_file):
        with open(data_file, 'r') as f:
            lines = f.readlines()
        lis = []
        for line in lines:
            l = [int(s) for s in list(line.strip().split())]
            lis.append(l) 
        return lis

> ##  Roll Out

In [ ]:
class Rollout(object):
    """ Rollout Policy """

    def __init__(self, model, update_rate):
        self.ori_model = model
        self.own_model = copy.deepcopy(model)
        self.update_rate = update_rate

    def get_reward(self, x, num, discriminator):
        """
        Inputs: x, num, discriminator
            - x: (batch_size, seq_len) input data
            - num: rollout number
            - discriminator: discrimanator model
        """
        rewards = []
        batch_size = x.size(0)
        seq_len = x.size(1)
        for i in range(num):
            for l in range(1, seq_len):
                data = x[:, 0:l]
                samples = self.own_model.sample(batch_size, seq_len, data)
                pred = discriminator(samples)
                pred = pred.cpu().data[:,1].numpy()
                if i == 0:
                    rewards.append(pred)
                else:
                    rewards[l-1] += pred

            # for the last token
            pred = discriminator(x)
            pred = pred.cpu().data[:, 1].numpy()
            if i == 0:
                rewards.append(pred)
            else:
                rewards[seq_len-1] += pred
        rewards = np.transpose(np.array(rewards)) / (1.0 * num) # batch_size * seq_len
        return rewards

    def update_params(self):
        dic = {}
        for name, param in self.ori_model.named_parameters():
            dic[name] = param.data
        for name, param in self.own_model.named_parameters():
            if name.startswith('emb'):
                param.data = dic[name]
            else:
                param.data = self.update_rate * param.data + (1 - self.update_rate) * dic[name]

# SeqGAN

In [26]:
# Arguemnts
parser = argparse.ArgumentParser(description='SeqGAN')
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
parser.add_argument('--hpc', action='store_true', default=False,
                    help='set to hpc mode')
parser.add_argument('--data_path', type=str, default='/content/data/', metavar='PATH',
                    help='data path to save files (default: /content/data/)')
parser.add_argument('--rounds', type=int, default=150, metavar='N',
                    help='rounds of adversarial training (default: 150)')
parser.add_argument('--g_pretrain_steps', type=int, default=120, metavar='N',
                    help='steps of pre-training of generators (default: 120)')
parser.add_argument('--d_pretrain_steps', type=int, default=50, metavar='N',
                    help='steps of pre-training of discriminators (default: 50)')
parser.add_argument('--g_steps', type=int, default=1, metavar='N',
                    help='steps of generator updates in one round of adverarial training (default: 1)')
parser.add_argument('--d_steps', type=int, default=3, metavar='N',
                    help='steps of discriminator updates in one round of adverarial training (default: 3)')
parser.add_argument('--gk_epochs', type=int, default=1, metavar='N',
                    help='epochs of generator updates in one step of generate update (default: 1)')
parser.add_argument('--dk_epochs', type=int, default=3, metavar='N',
                    help='epochs of discriminator updates in one step of discriminator update (default: 3)')
parser.add_argument('--update_rate', type=float, default=0.8, metavar='UR',
                    help='update rate of roll-out model (default: 0.8)')
parser.add_argument('--n_rollout', type=int, default=16, metavar='N',
                    help='number of roll-out (default: 16)')
parser.add_argument('--vocab_size', type=int, default=11292, metavar='N',
                    help='vocabulary size (default: 11292)')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size (default: 64)')
parser.add_argument('--n_samples', type=int, default=6400, metavar='N',
                    help='number of samples gerenated per time (default: 6400)')
parser.add_argument('--gen_lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate of generator optimizer (default: 1e-3)')
parser.add_argument('--dis_lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate of discriminator optimizer (default: 1e-3)')
parser.add_argument('--no_cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')

#########################################################################################
#  Generator  Hyper-parameters
######################################################################################
g_embed_dim = 30 # embedding dimension (pretrained: 200, pk: 30)
g_hidden_dim = 300 # hidden state dimension of lstm cell
g_seq_len = 30 # sequence length
START_TOKEN = 0
PRE_EPOCH_NUM = 120  # supervise (maximum likelihood estimation) epochs
SEED = 88
BATCH_SIZE = 64

#EMB_DIM = 32 # embedding dimension

#########################################################################################
#  Discriminator  Hyper-parameters
#########################################################################################
d_num_class = 2
d_embed_dim = 30
d_filter_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 30]
d_num_filters = [100, 200, 200, 200, 200, 100, 100, 100, 100, 100, 160, 160]
d_dropout_prob = 0.75
dis_l2_reg_lambda = 0.2
dis_batch_size = 64

#########################################################################################
#  Basic Training Parameters
#########################################################################################
TOTAL_BATCH = 200
generated_num = 100
sample_num = 10

# original seqgan parameter
# HIDDEN_DIM = 32
# PRE_EPOCH_NUM = 120
# TOTAL_BATCH = 200
# generated_num = 10000

POSITIVE_FILE = './data/3_pk_data_index.txt'
NEGATIVE_FILE = 'save/negative_sample.txt'
eval_file = 'save/eval_file.txt'
# "pretrain" or "poke"
embed_flag = "poke"

a = open('./data/3_pk_data_index.pkl', 'rb')
real_data = pickle.load(a)

a = open('./data/pk_pos2idx.pkl', 'rb')
vocab_to_int = pickle.load(a)

a = open('./data/pk_idx2pos.pkl', 'rb')
int_to_vocab = pickle.load(a)
print(int_to_vocab)

if embed_flag == "pretrain":
    a = open('./data/pretrain_embedding_vec.pkl', 'rb')
elif embed_flag == "poke":
    a = open('./data/pk_embedding_vec.pkl', 'rb')
word_embedding_matrix = pickle.load(a)
word_embedding_matrix = word_embedding_matrix.astype(np.float32)

# a = open('./data/word_dict.pickle', 'rb')
# word_dict = pickle.load(a)

real_data_vocab = [[int_to_vocab[i] for i in sample if int_to_vocab[i] != '<PAD>'] for sample in real_data]
real_data_vocab = [' '.join(sample) for sample in real_data_vocab]
print(len(real_data_vocab))


def generate_samples(model, batch_size, generated_num, output_file):
    samples = []
    for _ in range(int(generated_num / batch_size)):
        sample = model.sample(batch_size, g_seq_len).cpu().data.numpy().tolist()
        samples.extend(sample)
    with open(output_file, 'w') as fout:
        for sample in samples:
            string = ' '.join([str(s) for s in sample])
     
            fout.write('{}\n'.format(string))


def train_generator_MLE(gen, data_iter, criterion, optimizer, epochs, 
        gen_pretrain_train_loss, args):
    """
    Train generator with MLE
    """
    for epoch in range(epochs):
        total_loss = 0.
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = gen(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        data_iter.reset()
    avg_loss = total_loss / len(data_iter)
    print("Epoch {}, train loss: {:.5f}".format(epoch, avg_loss))
    gen_pretrain_train_loss.append(avg_loss)

def train_generator_PG(gen, dis, rollout, pg_loss, optimizer, epochs, args):
    """
    Train generator with the guidance of policy gradient
    """
    for epoch in range(epochs):
        # construct the input to the genrator, add zeros before samples and delete the last column
        samples = generator.sample(args.batch_size, g_seq_len)
        zeros = torch.zeros(args.batch_size, 1, dtype=torch.int64)
        if samples.is_cuda:
            zeros = zeros.cuda()
        inputs = torch.cat([zeros, samples.data], dim = 1)[:, :-1].contiguous()
        targets = samples.data.contiguous().view((-1,))

        # calculate the reward
        rewards = torch.tensor(rollout.get_reward(samples, args.n_rollout, dis))
        if args.cuda:
            rewards = rewards.cuda()

        # update generator
        output = gen(inputs)
        loss = pg_loss(output, targets, rewards)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def eval_generator(model, data_iter, criterion, args):
    """
    Evaluate generator with NLL
    """
    total_loss = 0.
    with torch.no_grad():
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            pred = model(data)
            loss = criterion(pred, target)
            total_loss += loss.item()
    avg_loss = total_loss / len(data_iter)
    return avg_loss


def train_discriminator(dis, gen, criterion, optimizer, epochs, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args):
    """
    Train discriminator
    """
    generate_samples(gen, args.batch_size, args.n_samples, NEGATIVE_FILE)
    data_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
    for epoch in range(epochs):
        correct = 0
        total_loss = 0.
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = dis(data)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).cpu().sum()
            loss = criterion(output, target)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        data_iter.reset()
        avg_loss = total_loss / len(data_iter)
        acc = correct.item() / data_iter.data_num
        print("Epoch {}, train loss: {:.5f}, train acc: {:.3f}".format(epoch, avg_loss, acc))
        dis_adversarial_train_loss.append(avg_loss)
        dis_adversarial_train_acc.append(acc)


def eval_discriminator(model, data_iter, criterion, args):
    """
    Evaluate discriminator, dropout is enabled
    """
    correct = 0
    total_loss = 0.
    with torch.no_grad():
        for data, target in data_iter:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            target = target.contiguous().view(-1)
            output = model(data)
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).cpu().sum()
            loss = criterion(output, target)
            total_loss += loss.item()
    avg_loss = total_loss / len(data_iter)
    acc = correct.item() / data_iter.data_num
    return avg_loss, acc


def adversarial_train(gen, dis, rollout, pg_loss, nll_loss, gen_optimizer, dis_optimizer, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args):
    """
    Adversarially train generator and discriminator
    """
    # train generator for g_steps
    print("#Train generator")
    for i in range(args.g_steps):
        print("##G-Step {}".format(i))
        train_generator_PG(gen, dis, rollout, pg_loss, gen_optimizer, args.gk_epochs, args)

    # train discriminator for d_steps
    print("#Train discriminator")
    for i in range(args.d_steps):
        print("##D-Step {}".format(i))
        train_discriminator(dis, gen, nll_loss, dis_optimizer, args.dk_epochs, 
            dis_adversarial_train_loss, dis_adversarial_train_acc, args)

    # update roll-out model
    rollout.update_params()


def make_sample(eval_file, sample_num):
    samples = []
    with open(eval_file, 'r') as f:
        for line in f:
            line = line.strip()
            line = line.split()
            parse_line = [x for x in line]
            samples.append(parse_line)

    sample_vocab = samples

    return sample_vocab


# if __name__ == '__main__':


#     # Parse arguments
#     args = parser.parse_args()
#     args.cuda = not args.no_cuda and torch.cuda.is_available()
#     torch.manual_seed(args.seed)
#     if args.cuda:
#         torch.cuda.manual_seed(args.seed)
#     if not args.hpc:
#         args.data_path = ''

#     POSITIVE_FILE = args.data_path + POSITIVE_FILE
#     NEGATIVE_FILE = args.data_path + NEGATIVE_FILE

#     # Set models, criteria, optimizers
#     generator = Generator(args.vocab_size, g_embed_dim, g_hidden_dim, args.cuda)
#     discriminator = Discriminator(d_num_class, args.vocab_size, d_embed_dim, d_filter_sizes, d_num_filters, d_dropout_prob)
#     target_lstm = TargetLSTM(args.vocab_size, g_embed_dim, g_hidden_dim, args.cuda)


#     nll_loss = nn.NLLLoss()
#     pg_loss = PGLoss()


#     if args.cuda:
#         generator = generator.cuda()
#         discriminator = discriminator.cuda()
#         target_lstm = target_lstm.cuda()
#         nll_loss = nll_loss.cuda()
#         pg_loss = pg_loss.cuda()
#         cudnn.benchmark = True
#     gen_optimizer = optim.Adam(params=generator.parameters(), lr=args.gen_lr)
#     dis_optimizer = optim.SGD(params=discriminator.parameters(), lr=args.dis_lr)

#     # Container of experiment data
#     gen_pretrain_train_loss = []
#     gen_pretrain_eval_loss = []
#     dis_pretrain_train_loss = []
#     dis_pretrain_train_acc = []
#     dis_pretrain_eval_loss = []
#     dis_pretrain_eval_acc = []
#     gen_adversarial_eval_loss = []
#     dis_adversarial_train_loss = []
#     dis_adversarial_train_acc = []
#     dis_adversarial_eval_loss = []
#     dis_adversarial_eval_acc = []

#     # Generate toy data using target LSTM
#     print('#####################################################')
#     print('Generating data ...')
#     print('#####################################################\n\n')
#     generate_samples(target_lstm, args.batch_size, args.n_samples, POSITIVE_FILE)

#     # Pre-train generator using MLE
#     print('#####################################################')
#     print('Start pre-training generator with MLE...')
#     print('#####################################################\n')
#     gen_data_iter = GenDataIter(POSITIVE_FILE, args.batch_size)
#     for i in range(args.g_pretrain_steps):
#     #for i in range(1):      
#         print("G-Step {}".format(i))
#         train_generator_MLE(generator, gen_data_iter, nll_loss, 
#             gen_optimizer, args.gk_epochs, 
#             gen_pretrain_train_loss, args)
#         generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
#         eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
#         gen_loss = eval_generator(target_lstm, eval_iter, nll_loss, args)
#         gen_pretrain_eval_loss.append(gen_loss)
#         print("eval loss: {:.5f}\n".format(gen_loss))
#     print('#####################################################\n\n')

#     # Pre-train discriminator
#     print('#####################################################')
#     print('Start pre-training discriminator...')
#     print('#####################################################\n')
#     #for i in range(1):
#     for i in range(args.d_pretrain_steps):      
#         print("D-Step {}".format(i))
#         train_discriminator(discriminator, generator, nll_loss, 
#             dis_optimizer, args.dk_epochs, 
#             dis_adversarial_train_loss, dis_adversarial_train_acc, args)
#         generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
#         eval_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
#         dis_loss, dis_acc = eval_discriminator(discriminator, eval_iter, nll_loss, args)
#         dis_pretrain_eval_loss.append(dis_loss)
#         dis_pretrain_eval_acc.append(dis_acc)
#         print("eval loss: {:.5f}, eval acc: {:.3f}\n".format(dis_loss, dis_acc))
#     print('#####################################################\n\n')

#     # Adversarial training
#     print('#####################################################')
#     print('Start adversarial training...')
#     print('#####################################################\n')
#     rollout = Rollout(generator, args.update_rate)
#     #for i in range(1):    
#     for i in range(args.rounds):
#         print("Round {}".format(i))
#         adversarial_train(generator, discriminator, rollout, 
#             pg_loss, nll_loss, gen_optimizer, dis_optimizer, 
#             dis_adversarial_train_loss, dis_adversarial_train_acc, args)
#         generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
#         gen_eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
#         dis_eval_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
#         gen_loss = eval_generator(target_lstm, gen_eval_iter, nll_loss, args)
#         gen_adversarial_eval_loss.append(gen_loss)
#         dis_loss, dis_acc = eval_discriminator(discriminator, dis_eval_iter, nll_loss, args)
#         dis_adversarial_eval_loss.append(dis_loss)
#         dis_adversarial_eval_acc.append(dis_acc)
#         print("gen eval loss: {:.5f}, dis eval loss: {:.5f}, dis eval acc: {:.3f}\n"
#             .format(gen_loss, dis_loss, dis_acc))

#     # Save experiment data
#     with open(args.data_path + 'experiment.pkl', 'wb') as f:
#         pickle.dump(
#             (gen_pretrain_train_loss,
#                 gen_pretrain_eval_loss,
#                 dis_pretrain_train_loss,
#                 dis_pretrain_train_acc,
#                 dis_pretrain_eval_loss,
#                 dis_pretrain_eval_acc,
#                 gen_adversarial_eval_loss,
#                 dis_adversarial_train_loss,
#                 dis_adversarial_train_acc,
#                 dis_adversarial_eval_loss,
#                 dis_adversarial_eval_acc),
#             f,
#             protocol=pickle.HIGHEST_PROTOCOL
#         )





{0: '<start>', 1: '하', 2: '이', 3: '의', 4: '는', 5: '다', 6: '에', 7: '을', 8: '은', 9: '되', 10: '를', 11: '어', 12: '고', 13: '있', 14: '가', 15: '아', 16: '으로', 17: '로', 18: '에서', 19: '년', 20: '과', 21: '들', 22: '도', 23: '지', 24: '일', 25: '것', 26: '그', 27: '게', 28: '와', 29: '오', 30: '기', 31: '였', 32: '늘', 33: '여', 34: '수', 35: '등', 36: '며', 37: '더', 38: '나', 39: '시', 40: '라', 41: '주', 42: '리', 43: '대하', 44: '부터', 45: '에게', 46: '때', 47: '어서', 48: '만', 49: '면', 50: '저', 51: '보', 52: '세', 53: '없', 54: '후', 55: '키', 56: '지만', 57: '자', 58: '중', 59: '까지', 60: '따르', 61: '라는', 62: '라고', 63: '두', 64: '위', 65: '니', 66: '대', 67: '타', 68: '전', 69: '경우', 70: '가지', 71: '적', 72: '이후', 73: '알', 74: '한', 75: '마', 76: '사람', 77: '때문', 78: '사', 79: '서', 80: '미', 81: '많', 82: '성', 83: '군', 84: '만들', 85: '개', 86: '바', 87: '아니', 88: '제', 89: '또는', 90: '이름', 91: '레', 92: '코', 93: '이나', 94: '말', 95: '다른', 96: '우', 97: '티', 98: '노', 99: '김', 100: '현재', 101: '안', 102: '내', 103: '정', 104: '또한', 105: '히', 106: '차', 107: '데',

# **V. SeqGAN 메인 실행문**

> ## 세팅

In [63]:


# Parse arguments
args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
if not args.hpc:
    args.data_path = ''

# Positive_File / Negative_File ?
POSITIVE_FILE = args.data_path + POSITIVE_FILE
NEGATIVE_FILE = args.data_path + NEGATIVE_FILE



> ## 모델/로스 생성

In [64]:
   
   
# Set models, criteria, optimizers
generator = Generator(args.vocab_size, g_embed_dim, g_hidden_dim, args.cuda)
discriminator = Discriminator(d_num_class, args.vocab_size, d_embed_dim, d_filter_sizes, d_num_filters, d_dropout_prob)
target_lstm = TargetLSTM(args.vocab_size, g_embed_dim, g_hidden_dim, args.cuda)

nll_loss = nn.NLLLoss()
pg_loss = PGLoss()



> ## cuda / optimizer 생성

In [66]:

if args.cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    target_lstm = target_lstm.cuda()
    nll_loss = nll_loss.cuda()
    pg_loss = pg_loss.cuda()
    cudnn.benchmark = True

gen_optimizer = optim.Adam(params=generator.parameters(), lr=args.gen_lr)
dis_optimizer = optim.SGD(params=discriminator.parameters(), lr=args.dis_lr)


> ## Pretrain Generator/Discriminator

In [67]:
# Container of experiment data
gen_pretrain_train_loss = []
gen_pretrain_eval_loss = []
dis_pretrain_train_loss = []
dis_pretrain_train_acc = []
dis_pretrain_eval_loss = []
dis_pretrain_eval_acc = []
gen_adversarial_eval_loss = []
dis_adversarial_train_loss = []
dis_adversarial_train_acc = []
dis_adversarial_eval_loss = []
dis_adversarial_eval_acc = []

In [71]:
%%time

MODEL_PATH = "/content/save/generator.pt"

# Generate toy data using target LSTM
print('#####################################################')
print('Generating data ...')
print('#####################################################\n\n')
generate_samples(target_lstm, args.batch_size, args.n_samples, POSITIVE_FILE)

# Pre-train generator using MLE
print('#####################################################')
print('Start pre-training generator with MLE...')
print('#####################################################\n')
gen_data_iter = GenDataIter(POSITIVE_FILE, args.batch_size)

#for i in range(args.g_pretrain_steps):
for i in range(30):      
    print("G-Step {}".format(i))
    train_generator_MLE(generator, gen_data_iter, nll_loss, 
        gen_optimizer, args.gk_epochs, 
        gen_pretrain_train_loss, args)
    generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
    eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
    gen_loss = eval_generator(target_lstm, eval_iter, nll_loss, args)
    gen_pretrain_eval_loss.append(gen_loss)
    print("eval loss: {:.5f}\n".format(gen_loss))
print('#####################################################\n\n')

print("Pretrain Generator Model Save...!")
torch.save(generator , MODEL_PATH)

# Pre-train discriminator
print('#####################################################')
print('Start pre-training discriminator...')
print('#####################################################\n')
for i in range(5):
#for i in range(args.d_pretrain_steps):      
    print("D-Step {}".format(i))
    train_discriminator(discriminator, generator, nll_loss, 
        dis_optimizer, args.dk_epochs, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args)
    generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
    eval_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
    dis_loss, dis_acc = eval_discriminator(discriminator, eval_iter, nll_loss, args)
    dis_pretrain_eval_loss.append(dis_loss)
    dis_pretrain_eval_acc.append(dis_acc)
    print("eval loss: {:.5f}, eval acc: {:.3f}\n".format(dis_loss, dis_acc))
print('#####################################################\n\n')

#


#####################################################
Generating data ...
#####################################################


#####################################################
Start pre-training generator with MLE...
#####################################################

G-Step 0
Epoch 0, train loss: 7.94234
eval loss: 25.23042

G-Step 1
Epoch 0, train loss: 7.94234
eval loss: 25.22309

G-Step 2
Epoch 0, train loss: 7.94234
eval loss: 25.21256

G-Step 3
Epoch 0, train loss: 7.94234
eval loss: 25.25578

G-Step 4
Epoch 0, train loss: 7.94234
eval loss: 25.24686

G-Step 5
Epoch 0, train loss: 7.94234
eval loss: 25.25230

G-Step 6
Epoch 0, train loss: 7.94234
eval loss: 25.25003

G-Step 7
Epoch 0, train loss: 7.94234
eval loss: 25.22532

G-Step 8
Epoch 0, train loss: 7.94234
eval loss: 25.25004

G-Step 9
Epoch 0, train loss: 7.94234
eval loss: 25.24168

G-Step 10
Epoch 0, train loss: 7.94234
eval loss: 25.24952

G-Step 11
Epoch 0, train loss: 7.94234
eval loss: 25.25326

G-Step 12


In [72]:

generator = torch.load(MODEL_PATH)

generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
gen_loss = eval_generator(target_lstm, eval_iter, nll_loss, args)
gen_pretrain_eval_loss.append(gen_loss)
print("eval loss: {:.5f}\n".format(gen_loss))

translating_sample(GENERATE_PATH , VOCA_PATH)

eval loss: 25.29077

생성된 데이터 확인.....
생성된 가사 수 : 6400 , 가사 시퀀스 수 : 30 
 
저장된 단어사전 확인....
단어 사전에 등록된 단어 수 :  11292
 
whatever 오던 넘칠 불어 간주 비치는 고맙던 되돌려질 묻은 까만 hearts 없다 시큰둥 다져질 닫지만 삭여주려 벗었네 갈수록 변하 칭얼대면서 여기는 가려주는 않으리라는 처음처럼 복 알아보는 발버둥 간절히 젊음 나면 
------------------------
새로운 appear 커텐 않을까 아마도 好 지나가리라 들지 고장 주사 보다가도 volume 필요해요 학교 needs 반의 build 오늘 있었는걸 나였으면해 되어주면 get 때르릉 소린 달리아 떠나네 그려가리 늦었지만 호숫가 차오르는데 
------------------------
으론 appear 간절히 비춰줄 비탄 好 Elliott 하였다 what 시들었지 애길 쏙 고래 깨네요 fella 왔건만 속삭이며 복도 들려줘 신발 샌가 어설펐던 으론 꺼지고 있길 Destiny 물가 소나기 <start> <start> 
------------------------
으음 현실 달라 수면 갖고 힘들었었던 버겁기만 뛰어 가에 머무르진 들킨 날리는 앉자마자 Wait 비추지 god 待 죽을만큼 끊어줄 둬 살이 부자 길기만 지내 들이죠 떠나가줘 rap 후회되는 깨면 앞길 
------------------------
노스 으론 Halloween 살이 마다 않았지 leaving 벗어 sharry 즐거웠던 잘못 만의 윤오 몰골 꺼내는 있었니 찾아낸 돛 자리는 만배 해본다 어떡하지 총동원 이겠죠 걸어온 nights 鼓動止 행복하다 두려웠기에 <start> 
------------------------
새로운 시인 호숫가 않을까 asi 보내 맡긴다 fantasy 싣고 조심스러워 내밀다니 잘해줄 기대어요 떼러 들던 막히 금방 좋았었는지 달리듯 forget 할라면 랑 만으로 선명해요 잠기 잊어도 없다고 마이 들리면 die

In [50]:


def translating_sample(sample_path , voca_path):

  print("================================================")
  print("생성된 데이터 확인.....")
  df = pd.read_csv(sample_path , names = ["data"] , sep = "\t" )
  Sentences = [sentence.split(" ") for sentence in df['data'].values]
  print("생성된 가사 수 : {} , 가사 시퀀스 수 : {} ".format(len(Sentences) , len(Sentences[0])))
  print(" ")

  print("저장된 단어사전 확인....")
  a = open(voca_path, 'rb')
  voca = pickle.load(a)
  print("단어 사전에 등록된 단어 수 : " , len(voca))
  print(" ")

  # 확인할 무작위 시드 / 사이즈
  random_seed = 100
  random_size = 10

  test_Sentences = Sentences[random_seed:random_seed + random_size]


  for sentence in test_Sentences:
    real_sentence = ""
    for s in sentence:

      try:
        s = int(s)
      except: pass

      if voca.get(s):
        if voca[s] == "UNK":
          break
        real_sentence += voca[s] + " "

    print(real_sentence)
    print("------------------------")


In [53]:
VOCA_PATH = "/content/data/pk_idx2pos.pkl"
GENERATE_PATH = "/content/save/negative_sample.txt"

translating_sample(GENERATE_PATH , VOCA_PATH)


생성된 데이터 확인.....
생성된 가사 수 : 6400 , 가사 시퀀스 수 : 30 
 
저장된 단어사전 확인....
단어 사전에 등록된 단어 수 :  11292
 
노스 으론 삼아 크림 지우고 앞날 탄산 양 믿어줘요 단칸 쓸까 늦봄 그러려고 해주렴 그랬나는 당당히 부신 차오르는데 닿고 소모 바닥났네 상관없이도 알았을 흩어 초라할 시련 떠올린다 골인 떠들어 <start> 
------------------------
새로운 appear 이라고 모른 또 어색했었기에 자장가 설렘도 슬픈가요 댈테니 할꺼야 기회 번만 작일 가야 말거예요 기댔지 도와주지 뒀던 수동 쳐도 재촉 나완 탐내지 지킨 숱 세로 최 부끄러운 만드 
------------------------
노스 시인 고 단단한 울었던가요 돌아올까 친해질까 상치 달려가 시들어 부를 소용없잖아 갔죠 다가가게 넘치면 잔 먼 변하지 골라줘 최고 멈췄고 보내준다고 끝난 햇님 거고 부드런 놓여 이뤄지 들 전분 
------------------------
깬 그렇게도 하지말란 鏡 보내온 아니더라도 바라보다가 어떡해야 돌아 째 술 분명 향연 잘못 둥지 서든 묻어주던 벗 먼저 배고파도 이다 하랴 껴진 주나요 새카맣 적당한 조금 않았다면 끝난 <start> 
------------------------
실감 정적 입은 고해 생각나 오래도록 호숫가 자리는 공부 할꺼야 never 처량하구나 안쓰러 더위 오르다 순식간 모를거 차오를 다쳐도 골목 And Soul 움큼 수목원 삼겹살 들려주고 기다리던 들려오고 자유 해지 
------------------------
새로운 보이 따라온 연락 외친다 보죠 길지라도 거미줄 않아도 놀랍지 젖어 말씀 충분 떠났니 같아 spring 전이 재잘대던 불러오라고 피치 스치던 대신 무너질지 부끄런 뒤엉킨 담은 울컥 않을거에요 그리하겠소 평화로 
------------------------
깬 그렇게도 하지말란 鏡 자전거 있습니다 풀꽃 붙인 살아가도록 느껴가고 건배 돌아오지 정해진 있기를 했음 자체 

> ## Adversarial Training

In [ ]:

# Adversarial training
print('#####################################################')
print('Start adversarial training...')
print('#####################################################\n')
rollout = Rollout(generator, args.update_rate)
#for i in range(1):    
for i in range(args.rounds):
    print("Round {}".format(i))
    adversarial_train(generator, discriminator, rollout, 
        pg_loss, nll_loss, gen_optimizer, dis_optimizer, 
        dis_adversarial_train_loss, dis_adversarial_train_acc, args)
    generate_samples(generator, args.batch_size, args.n_samples, NEGATIVE_FILE)
    gen_eval_iter = GenDataIter(NEGATIVE_FILE, args.batch_size)
    dis_eval_iter = DisDataIter(POSITIVE_FILE, NEGATIVE_FILE, args.batch_size)
    gen_loss = eval_generator(target_lstm, gen_eval_iter, nll_loss, args)
    gen_adversarial_eval_loss.append(gen_loss)
    dis_loss, dis_acc = eval_discriminator(discriminator, dis_eval_iter, nll_loss, args)
    dis_adversarial_eval_loss.append(dis_loss)
    dis_adversarial_eval_acc.append(dis_acc)
    print("gen eval loss: {:.5f}, dis eval loss: {:.5f}, dis eval acc: {:.3f}\n"
        .format(gen_loss, dis_loss, dis_acc))

# Save experiment data
with open(args.data_path + 'experiment.pkl', 'wb') as f:
    pickle.dump(
        (gen_pretrain_train_loss,
            gen_pretrain_eval_loss,
            dis_pretrain_train_loss,
            dis_pretrain_train_acc,
            dis_pretrain_eval_loss,
            dis_pretrain_eval_acc,
            gen_adversarial_eval_loss,
            dis_adversarial_train_loss,
            dis_adversarial_train_acc,
            dis_adversarial_eval_loss,
            dis_adversarial_eval_acc),
        f,
        protocol=pickle.HIGHEST_PROTOCOL
    )

